In [40]:
from PyPDF2 import PdfReader
import glob

for bula in glob.glob("dataset/*.pdf"):
    reader = PdfReader(bula)
    text = ""
    for page in reader.pages:
        text+=page.extract_text()
        text = text.replace("\n", "").lower()
    print(text[:2000])
    print("\n\n\n")

          miorrelax®  (dipirona monoidratada + citrato de orfenadrina + cafeína)    brainfarma indústria química e farmacêutica s.a.  comprimido 300mg + 35mg + 50mg  miorrelax ®- comprimido - bula para o paciente   1 i - identificação do medicamento:  miorrelax ® dipirona monoidratada + citrato de orfenadrina + ca feína   apresentações comprimido. embalagens contendo 20, 30 ou 200 comprimidos.  via de administração: oral  uso adulto   composição cada comprimido contém: dipirona  monoidratada............................. ................................................... .............. 300mg citrato de orfenadrina (equivalente a 20,4mg de orf enadrina)......................................35mg   cafeína............................................ ................................................... ........................... 50mg excipientes q.s.p.................................. ................................................... ........ 1 comprimido (povidona, metabissulfito de sód

In [30]:
%%capture

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

import transformers
from transformers import AutoTokenizer
from transformers import  DistilBertForTokenClassification

from torch.optim import AdamW

import torch
import torch.nn as nn
from torch.optim import SGD
import torch.nn.functional as F
from torch.utils.data import DataLoader

from sklearn.metrics import accuracy_score,f1_score, precision_score, recall_score

from spacy.training import offsets_to_biluo_tags, biluo_to_iob


In [42]:
nlp = spacy.load("pt_core_news_sm")

df = pd.read_json("dataset/bula_annotation_ner.jsonl", lines=True)
df.rename(columns = {'text':'sentence'}, inplace = True)
df["tags"] = df.apply(lambda row: " ".join(biluo_to_iob(offsets_to_biluo_tags(nlp.make_doc(row["sentence"]), row["label"]))), axis=1)
df

,id,sentence,label,Comments,tags
0,6,Cada comprimido contém 300 mg de dipirona mon...,"[[34, 55, Ingredient], [66, 88, Ingredient], [...",[],O O O O O O O B-Ingredient I-Ingredient O O O ...
1,7,DORFLEX é indicado no alívio da dor associada...,"[[0, 7, Drug], [23, 36, Use], [49, 71, Use], [...",[],B-Drug O O O O B-Use I-Use I-Use O O B-Use I-U...
2,8,DORFLEX possui ação analgésica e relaxante mus...,"[[0, 7, Drug], [20, 30, Use], [33, 51, Use]]",[],B-Drug O O B-Use O B-Use I-Use O
3,9,composição cada comprimido contém: ácido aceti...,"[[35, 57, Ingredient], [64, 71, Ingredient]]",[],O O O O O B-Ingredient I-Ingredient O O B-Ingr...
4,10,doril é indicado como analgésico e antipirétic...,"[[0, 5, Drug], [22, 32, Use], [35, 47, Use], [...",[],B-Drug O O O B-Use O B-Use O O O O O O O B-Use...
5,11,este medicamento é um analgésico com ação anti...,"[[22, 32, Use], [42, 54, Use], [83, 86, Use], ...",[],O O O O B-Use O O B-Use O O O O O O B-Use O O ...
6,12,apresentações comprimidos contendo 500 mg d...,"[[48, 71, Ingredient]]",[],O O O O O O O O O B-Ingredient I-Ingredient I-...
7,13,aspirina está indicada para: o alívio sintom...,"[[0, 8, Drug], [56, 61, Use], [103, 116, Use],...",[],B-Drug O O O O O O O O O O O B-Use O O O O O O...
8,14,aspirina contém a substância ativa ácido a...,"[[0, 8, Drug], [38, 61, Ingredient], [104, 122...",[],B-Drug O O O O O O O B-Ingredient I-Ingredient...
9,24,composição cada comprimido contém: dipirona m...,"[[35, 57, Ingredient], [64, 86, Ingredient], [...",[],O O O O O B-Ingredient I-Ingredient I-Ingredie...


In [43]:
df_train, df_dev, df_test = np.split(df.sample(frac=1, random_state=42),
                            [int(.8 * len(df)), int(.9 * len(df))])

In [18]:
class DistilbertNER(nn.Module):
  """
  Implement NN class based on distilbert pretrained from Hugging face.
  Inputs : 
    tokens_dim : int specifyng the dimension of the classifier
  """
  
  def __init__(self, tokens_dim):
    super(DistilbertNER,self).__init__()
    
    if type(tokens_dim) != int:
            raise TypeError('Please tokens_dim should be an integer')

    if tokens_dim <= 0:
          raise ValueError('Classification layer dimension should be at least 1')

    self.pretrained = DistilBertForTokenClassification.from_pretrained("distilbert-base-uncased", num_labels = tokens_dim) #set the output of each token classifier = unique_lables


  def forward(self, input_ids, attention_mask, labels = None): #labels are needed in order to compute the loss
    """
  Forwad computation of the network
  Input:
    - inputs_ids : from model tokenizer
    - attention :  mask from model tokenizer
    - labels : if given the model is able to return the loss value
  """

    #inference time no labels
    if labels == None:
      out = self.pretrained(input_ids = input_ids, attention_mask = attention_mask )
      return out

    out = self.pretrained(input_ids = input_ids, attention_mask = attention_mask , labels = labels)
    return out

In [20]:
class NerDataset(torch.utils.data.Dataset):
  """
  Custom dataset implementation to get (text,labels) tuples
  Inputs:
   - df : dataframe with columns [tags, sentence]
  """
  
  def __init__(self, df):
    if not isinstance(df, pd.DataFrame):
      raise TypeError('Input should be a dataframe')
    
    if "tags" not in df.columns or "sentence" not in df.columns:
      raise ValueError("Dataframe should contain 'tags' and 'sentence' columns")

     
    
    tags_list = [i.split() for i in df["tags"].values.tolist()]
    texts = df["sentence"].values.tolist()

    self.texts = [tokenizer(text, padding = "max_length", truncation = True, return_tensors = "pt") for text in texts]
    self.labels = [match_tokens_labels(text, tags) for text,tags in zip(self.texts, tags_list)]

  def __len__(self):
    return len(self.labels)

  def __getitem__(self, idx):
    batch_text = self.texts[idx]
    batch_labels = self.labels[idx]

    return batch_text, torch.LongTensor(batch_labels)

In [21]:
class MetricsTracking():
  """
  In order make the train loop lighter I define this class to track all the metrics that we are going to measure for our model.
  """
  def __init__(self):

    self.total_acc = 0
    self.total_f1 = 0
    self.total_precision = 0
    self.total_recall = 0

  def update(self, predictions, labels , ignore_token = -100):
    '''
    Call this function every time you need to update your metrics.
    Where in the train there was a -100, were additional token that we dont want to label, so remove them.
    If we flatten the batch its easier to access the indexed = -100
    '''  
    predictions = predictions.flatten()
    labels = labels.flatten()
    
    predictions = predictions[labels != ignore_token]
    labels = labels[labels != ignore_token]

    predictions = predictions.to("cpu")
    labels = labels.to("cpu")

    acc = accuracy_score(labels,predictions)
    f1 = f1_score(labels, predictions, average = "macro")
    precision = precision_score(labels, predictions, average = "macro")
    recall = recall_score(labels, predictions, average = "macro")

    self.total_acc  += acc
    self.total_f1 += f1
    self.total_precision += precision
    self.total_recall  += recall

  def return_avg_metrics(self,data_loader_size):
    n = data_loader_size
    metrics = {
        "acc": round(self.total_acc / n ,3), 
        "f1": round(self.total_f1 / n, 3), 
        "precision" : round(self.total_precision / n, 3), 
        "recall": round(self.total_recall / n, 3)
          }
    return metrics  

In [24]:
def tags_2_labels(tags : str, tag2idx : dict):
  '''
  Method that takes a list of tags and a dictionary mapping and returns a list of labels (associated).
  Used to create the "label" column in df from the "tags" column.
  '''
  return [tag2idx[tag] if tag in tag2idx else unseen_label for tag in tags.split()] 

def tags_mapping(tags_series : pd.Series):
  """
  tag_series = df column with tags for each sentence.
  Returns:
    - dictionary mapping tags to indexes (label)
    - dictionary mappign inedexes to tags
    - The label corresponding to tag 'O'
    - A set of unique tags ecountered in the trainind df, this will define the classifier dimension
  """

  if not isinstance(tags_series, pd.Series):
      raise TypeError('Input should be a padas Series')

  unique_tags = set()
  
  for tag_list in df_train["tags"]:
    for tag in tag_list.split():
      unique_tags.add(tag)


  tag2idx = {k:v for v,k in enumerate(sorted(unique_tags))}
  idx2tag = {k:v for v,k in tag2idx.items()}

  unseen_label = tag2idx["O"]

  return tag2idx, idx2tag, unseen_label, unique_tags

def match_tokens_labels(tokenized_input, tags, ignore_token = -100):
        '''
        Used in the custom dataset.
        -100 will be tha label used to match additional tokens like [CLS] [PAD] that we dont care about. 
        Inputs : 
          - tokenized_input : tokenizer over the imput text -> {input_ids, attention_mask}
          - tags : is a single label array -> [O O O O O O O O O O O O O O B-tim O]
        
        Returns a list of labels that match the tokenized text -> [-100, 3,5,6,-100,...]
        '''

        #gives an array [ None , 0 , 1 ,2 ,... None]. Each index tells the word of reference of the token
        word_ids = tokenized_input.word_ids()

        previous_word_idx = None
        label_ids = []

        for word_idx in word_ids:

            if word_idx is None:
                label_ids.append(ignore_token)

            #if its equal to the previous word we can add the same label id of the provious or -100 
            else :
                try:
                  reference_tag = tags[word_idx]
                  label_ids.append(tag2idx[reference_tag])
                except:
                  label_ids.append(ignore_token)
              
            
            previous_word_idx = word_idx

        return label_ids

def freeze_model(model,num_layers = 1):
  """
  Freeze last num_layers of a model to prevent ctastrophic forgetting.
  Doesn't seem to work weel, its better to fine tune the entire netwok
  """
  for id , params in enumerate(model.parameters()):
    if id == len(list(model.parameters())) - num_layers: 
      print("last layer unfreezed")
      params.requires_grad = True
    else:
      params.requires_grad = False
  return model

def train_loop(model, train_dataset, dev_dataset, optimizer,  batch_size, epochs):
  
  train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
  dev_dataloader = DataLoader(dev_dataset, batch_size = batch_size, shuffle = True)

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model = model.to(device)

  for epoch in range(epochs) : 
    
    train_metrics = MetricsTracking()
    total_loss_train = 0

    model.train() #train mode

    for train_data, train_label in tqdm(train_dataloader):

      train_label = train_label.to(device)
      '''
      squeeze in order to match the sizes. From [batch,1,seq_len] --> [batch,seq_len] 
      '''
      mask = train_data['attention_mask'].squeeze(1).to(device)
      input_id = train_data['input_ids'].squeeze(1).to(device)

      optimizer.zero_grad()
      
      output = model(input_id, mask, train_label)
      loss, logits = output.loss, output.logits
      predictions = logits.argmax(dim= -1) 

      #compute metrics
      train_metrics.update(predictions, train_label)
      total_loss_train += loss.item()

      #grad step
      loss.backward()
      optimizer.step()
    
    '''
    EVALUATION MODE
    '''            
    model.eval()

    dev_metrics = MetricsTracking()
    total_loss_dev = 0
    
    with torch.no_grad():
      for dev_data, dev_label in dev_dataloader:

        dev_label = dev_label.to(device)

        mask = dev_data['attention_mask'].squeeze(1).to(device)
        input_id = dev_data['input_ids'].squeeze(1).to(device)

        output = model(input_id, mask, dev_label)
        loss, logits = output.loss, output.logits

        predictions = logits.argmax(dim= -1)     

        dev_metrics.update(predictions, dev_label)
        total_loss_dev += loss.item()
    
    train_results = train_metrics.return_avg_metrics(len(train_dataloader))
    dev_results = dev_metrics.return_avg_metrics(len(dev_dataloader))

    print(f"TRAIN \nLoss: {total_loss_train / len(train_dataset)} \nMetrics {train_results}\n" ) 
    print(f"VALIDATION \nLoss {total_loss_dev / len(dev_dataset)} \nMetrics{dev_results}\n" )  

In [44]:
tag2idx, idx2tag , unseen_label, unique_tags = tags_mapping(df_train["tags"])

#create the label column from tag. Unseen labels will be tagged as "O"
for df in [df_train, df_dev, df_test]:
  df["labels"] = df["tags"].apply(lambda tags : tags_2_labels(tags, tag2idx))

In [45]:
#original text
text = df_train["sentence"].values.tolist()

#toeknized text
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
text_tokenized = tokenizer(text , padding = "max_length" , truncation = True, return_tensors = "pt" )

#mapping token to original word
word_ids = text_tokenized.word_ids()

In [46]:
model = DistilbertNER(len(unique_tags))
#Prevent Catastrofic Forgetting
#model = freeze_model(model, num_layers = 2)

#datasets
train_dataset = NerDataset(df_train)
dev_dataset = NerDataset(df_dev)

lr = 1e-2
optimizer = SGD(model.parameters(), lr=lr, momentum = 0.9)  


#MAIN
parameters = {
    "model": model,
    "train_dataset": train_dataset,
    "dev_dataset" : dev_dataset,
    "optimizer" : optimizer,
    "batch_size" : 32,
    "epochs" : 8
}

train_loop(**parameters)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [03:07<00:00, 187.37s/it]


TRAIN 
Loss: 0.10366870855030261 
Metrics {'acc': 0.046, 'f1': 0.042, 'precision': 0.161, 'recall': 0.178}

VALIDATION 
Loss 0.7843663096427917 
Metrics{'acc': 0.546, 'f1': 0.178, 'precision': 0.143, 'recall': 0.238}



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [02:20<00:00, 140.73s/it]


TRAIN 
Loss: 0.07697614243156031 
Metrics {'acc': 0.739, 'f1': 0.149, 'precision': 0.145, 'recall': 0.163}

VALIDATION 
Loss 0.6706796288490295 
Metrics{'acc': 0.574, 'f1': 0.182, 'precision': 0.144, 'recall': 0.25}



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [02:09<00:00, 129.91s/it]


...

TRAIN 
Loss: 0.051494977976146494 
Metrics {'acc': 0.774, 'f1': 0.145, 'precision': 0.129, 'recall': 0.167}

VALIDATION 
Loss 0.8177715539932251 
Metrics{'acc': 0.574, 'f1': 0.182, 'precision': 0.144, 'recall': 0.25}



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [04:11<00:00, 251.04s/it]


TRAIN 
Loss: 0.04474032238910073 
Metrics {'acc': 0.774, 'f1': 0.145, 'precision': 0.129, 'recall': 0.167}

VALIDATION 
Loss 0.7266360521316528 
Metrics{'acc': 0.574, 'f1': 0.182, 'precision': 0.144, 'recall': 0.25}



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [02:08<00:00, 128.56s/it]


TRAIN 
Loss: 0.04494705325678775 
Metrics {'acc': 0.774, 'f1': 0.145, 'precision': 0.129, 'recall': 0.167}

VALIDATION 
Loss 0.7217835783958435 
Metrics{'acc': 0.574, 'f1': 0.182, 'precision': 0.144, 'recall': 0.25}

